<a href="https://colab.research.google.com/github/visiont3lab/machine-learning-course/blob/main/regression/Categorical_And_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Esempio categorical variables

* [Reference Notebook Categorical Features](https://towardsdatascience.com/encoding-categorical-features-21a2651a065c)
* [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/chronic_kidney_disease)
* [Kaggle Kidney Deasease Dataset](https://www.kaggle.com/mansoordaku/ckdisease/data)
* [Chronic Kidney Deasease download](https://matthew-brett.github.io/cfd2019/data/chronic_kidney_disease)
* [Kidney Study Github](https://github.com/liuy14/Kidney_Disease_Detection)

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("https://matthew-brett.github.io/cfd2019/data/ckd_clean.csv")
df.to_csv("dati.csv")
df

,Age,Blood Pressure,Specific Gravity,Albumin,Sugar,Red Blood Cells,Pus Cell,Pus Cell clumps,Bacteria,Blood Glucose Random,Blood Urea,Serum Creatinine,Sodium,Potassium,Hemoglobin,Packed Cell Volume,White Blood Cell Count,Red Blood Cell Count,Hypertension,Diabetes Mellitus,Coronary Artery Disease,Appetite,Pedal Edema,Anemia,Class
0,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,56.0,3.8,111.0,2.5,11.2,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,1
1,53.0,90.0,1.020,2.0,0.0,abnormal,abnormal,present,notpresent,70.0,107.0,7.2,114.0,3.7,9.5,29.0,12100.0,3.7,yes,yes,no,poor,no,yes,1
2,63.0,70.0,1.010,3.0,0.0,abnormal,abnormal,present,notpresent,380.0,60.0,2.7,131.0,4.2,10.8,32.0,4500.0,3.8,yes,yes,no,poor,yes,no,1
3,68.0,80.0,1.010,3.0,2.0,normal,abnormal,present,present,157.0,90.0,4.1,130.0,6.4,5.6,16.0,11000.0,2.6,yes,yes,yes,poor,yes,no,1
4,61.0,80.0,1.015,2.0,0.0,abnormal,abnormal,notpresent,notpresent,173.0,148.0,3.9,135.0,5.2,7.7,24.0,9200.0,3.2,yes,yes,yes,poor,yes,yes,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,140.0,49.0,0.5,150.0,4.9,15.7,47.0,6700.0,4.9,no,no,no,good,no,no,0
154,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,75.0,31.0,1.2,141.0,3.5,16.5,54.0,7800.0,6.2,no,no,no,good,no,no,0
155,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,100.0,26.0,0.6,137.0,4.4,15.8,49.0,6600.0,5.4,no,no,no,good,no,no,0
156,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,114.0,50.0,1.0,135.0,4.9,14.2,51.0,7200.0,5.9,no,no,no,good,no,no,0


In [3]:
# Find String unique data 
lista = ["Red Blood Cells","Pus Cell","Pus Cell clumps","Bacteria","Hypertension","Diabetes Mellitus", "Coronary Artery Disease", "Appetite", "Anemia"]
for el in lista:
  print( df[el].unique() )

['normal' 'abnormal']
['abnormal' 'normal']
['present' 'notpresent']
['notpresent' 'present']
['yes' 'no']
['no' 'yes']
['no' 'yes']
['poor' 'good']
['yes' 'no']


In [4]:
# One Hot encoding
df_pandas_dummy = pd.get_dummies(df, prefix_sep=' - ', drop_first=False)
df_pandas_dummy

,Age,Blood Pressure,Specific Gravity,Albumin,Sugar,Blood Glucose Random,Blood Urea,Serum Creatinine,Sodium,Potassium,Hemoglobin,Packed Cell Volume,White Blood Cell Count,Red Blood Cell Count,Class,Red Blood Cells - abnormal,Red Blood Cells - normal,Pus Cell - abnormal,Pus Cell - normal,Pus Cell clumps - notpresent,Pus Cell clumps - present,Bacteria - notpresent,Bacteria - present,Hypertension - no,Hypertension - yes,Diabetes Mellitus - no,Diabetes Mellitus - yes,Coronary Artery Disease - no,Coronary Artery Disease - yes,Appetite - good,Appetite - poor,Pedal Edema - no,Pedal Edema - yes,Anemia - no,Anemia - yes
0,48.0,70.0,1.005,4.0,0.0,117.0,56.0,3.8,111.0,2.5,11.2,32.0,6700.0,3.9,1,0,1,1,0,0,1,1,0,0,1,1,0,1,0,0,1,0,1,0,1
1,53.0,90.0,1.020,2.0,0.0,70.0,107.0,7.2,114.0,3.7,9.5,29.0,12100.0,3.7,1,1,0,1,0,0,1,1,0,0,1,0,1,1,0,0,1,1,0,0,1
2,63.0,70.0,1.010,3.0,0.0,380.0,60.0,2.7,131.0,4.2,10.8,32.0,4500.0,3.8,1,1,0,1,0,0,1,1,0,0,1,0,1,1,0,0,1,0,1,1,0
3,68.0,80.0,1.010,3.0,2.0,157.0,90.0,4.1,130.0,6.4,5.6,16.0,11000.0,2.6,1,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,1,0
4,61.0,80.0,1.015,2.0,0.0,173.0,148.0,3.9,135.0,5.2,7.7,24.0,9200.0,3.2,1,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,55.0,80.0,1.020,0.0,0.0,140.0,49.0,0.5,150.0,4.9,15.7,47.0,6700.0,4.9,0,0,1,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0
154,42.0,70.0,1.025,0.0,0.0,75.0,31.0,1.2,141.0,3.5,16.5,54.0,7800.0,6.2,0,0,1,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0
155,12.0,80.0,1.020,0.0,0.0,100.0,26.0,0.6,137.0,4.4,15.8,49.0,6600.0,5.4,0,0,1,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0
156,17.0,60.0,1.025,0.0,0.0,114.0,50.0,1.0,135.0,4.9,14.2,51.0,7200.0,5.9,0,0,1,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0


## Machine Learning Example

### Read Data and One Hot encoding (Categorical Issue)

In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv("https://matthew-brett.github.io/cfd2019/data/ckd_clean.csv")
df_pandas_dummy = pd.get_dummies(df, prefix_sep=' - ', drop_first=False)
Y = df_pandas_dummy.pop("Blood Glucose Random").values
X = df_pandas_dummy.values
print(Y.shape)
print(X.shape)

(158,)
(158, 34)


### Split Data

In [6]:
from sklearn.model_selection import train_test_split
# Shuffle --> mescolate 90 training -- 10%
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.20,shuffle=True, random_state=25)
print(X_train.shape)
print(X_test.shape)

(126, 34)
(32, 34)


### Without Pipeline

In [7]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib # pickle
from sklearn.decomposition import PCA

sc = StandardScaler()
X_train_n = sc.fit_transform(X_train)

pca = PCA(0.98)
X_train_pca = pca.fit_transform(X_train_n)

lm = LinearRegression()
lm.fit(X_train_pca,y_train)

#print(sc.mean_)
#print(sc.var_)
#print(lm.coef_)
#print(lm.intercept_)

joblib.dump(sc, "sc.pkl") 
joblib.dump(pca, "pca.pkl") 
joblib.dump(lm, "lm.pkl") 

# Deploy
sc = joblib.load("sc.pkl")
poly = joblib.load("pca.pkl")
lm = joblib.load("lm.pkl")

X_test_n = sc.transform(X_test)
X_test_pca = poly.transform(X_test_n)
y_hat_test = lm.predict(X_test_pca)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


### With Pipeline

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib # pickle

pipeline_preprocess = Pipeline([
    ("sc", StandardScaler()), # fit # mean, std
    ("pca", PCA(0.98)), # fit # mean, std
    ("linear-model", LinearRegression()) # trovare i parametri a0 a1 a2 
])
pipeline_preprocess.fit(X_train,y_train)
joblib.dump(pipeline_preprocess, "model.pkl") # pickle --> onnx 

# Predizione
y_hat_test_pipe = pipeline_preprocess.predict(X_test)

# Deploy
ms = joblib.load("model.pkl")
y_hat_test_pipe = ms.predict(X_test)

### Check

In [9]:
y_hat_test - y_hat_test_pipe

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

### Visualize

In [10]:
import plotly.graph_objects as go
# Visualizazione
x = np.linspace(start=0,stop=len(y_test),num=len(y_test))
fig = go.Figure()
fig.add_trace( go.Scatter( x=x, y=y_test.flatten(),mode="markers+lines", name="y_test") )
fig.add_trace( go.Scatter( x=x, y=y_hat_test.flatten(),mode="markers+lines", name="y_hat_test") )
fig.add_trace( go.Scatter( x=x, y=y_hat_test_pipe.flatten(),mode="markers+lines", name="y_hat_test_pipe") )
fig.update_layout(title_text="Results", hovermode="x")
fig.show()